# Counting Boats from Space - Part 2

In [ ]:
%reload_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
%matplotlib inline

## 1. Install, Import requirements

In [ ]:
import sys
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchvision
!{sys.executable} -m pip install plotly

In [ ]:
# Various utilities
import os
import json
import math
import numpy as np
import matplotlib.pyplot as plt
import skimage
import torch
from torch.utils.data import DataLoader

In [ ]:
from src.dataset import getImageSetDirectories, S2_Dataset, plot_dataset
from src.model import Model
from src.train import train, get_failures_or_success

In [ ]:
torch.cuda.is_available() # gpu support

## 2. Init K-Fold Dataset

In [ ]:
data_dir = "/home/jovyan/data" # data directory (path)
checkpoint_dir = "/home/jovyan/checkpoints"
bands = ['img_08', 'bg_ndwi']
test_size = 0.2

In [ ]:
train_list, val_list, fig = getImageSetDirectories(data_dir=os.path.join(data_dir, 'chips'), labels_filename=os.path.join(data_dir, 'labels.csv'),
                                                   band_list=bands, test_size=test_size, plot_coords=False, plot_class_imbalance=True, use_KFold=True, seed=123)
fig # mapbox plot train/val coordinates

In [ ]:
train_dataset = S2_Dataset(imset_dir=train_list[0], augment=True, labels_filename=os.path.join(data_dir,'labels.csv'))
val_dataset = S2_Dataset(imset_dir=val_list[0], augment=False, labels_filename=os.path.join(data_dir,'labels.csv'))
plot_dataset(train_dataset, n_frames=14, n_rows=2, cmap='gray')

## 4. Train PyTorch Classifier

In [ ]:
# training config
input_dim = train_dataset[0]['img'].shape[0]
hidden_dim, kernel_size, pool_size, n_max = 16, 3, 10, 1

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=16)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True, num_workers=4)

best_metrics = train(train_dataloader=train_dataloader, val_dataloader=val_dataloader,
             input_dim=input_dim, hidden_dim=hidden_dim, kernel_size=kernel_size, pool_size=pool_size, n_max=n_max, drop_proba=0.15,
             ld=0.2, water_ndwi=0.4,
             n_epochs=50, lr=0.007, lr_step=2, lr_decay=0.95,
             device='cpu', checkpoints_dir=checkpoint_dir, seed=42, verbose=1, version='0.0.2')

In [ ]:
# Epoch 72: train_clf_error 0.11907 / train_reg_error 0.17316 / val_clf_error 0.04312 / val_reg_error 0.07061

# Epoch 49: train_clf_error 0.11514 / train_reg_error 0.16339 / val_clf_error 0.11759 / val_reg_error 0.09096
# Epoch 45: train_clf_error 0.13395 / train_reg_error 0.17194 / val_clf_error 0.06827 / val_reg_error 0.05028
# Epoch 62: train_clf_error 0.11637 / train_reg_error 0.15685 / val_clf_error 0.02619 / val_reg_error 0.08079

In [ ]:
for k,v in best_metrics.items():
    print('{} {:.4f}'.format(k,v))

## 5. Test Model

In [ ]:
# load pretrained model
model = Model(input_dim=input_dim, hidden_dim=hidden_dim, kernel_size=kernel_size, pool_size=pool_size, n_max=n_max, device='cpu', version='0.0.1')
checkpoint_file = os.path.join(checkpoint_dir, model.folder, 'model.pth')
model.load_checkpoint(checkpoint_file=checkpoint_file)
model = model.eval()

In [ ]:
# Display failures (train, val), scatter plot (Predicted vs True) and inspect hidden channels --> Re label?
get_failures_or_success(model, val_dataset, hidden_channel=1, success=None, filter_on=None, plot_heatmap=False)